In [34]:
import numpy as np
import pandas as pd
from pandasql import sqldf 
from ast import literal_eval
import helper_functions as hf

## Reading files

In [2]:
# read in files needed for feature engineering
regular_season_compact_all = pd.read_csv('data/MRegularSeasonCompactResults.csv')
regular_season_detailed = pd.read_csv('data/MRegularSeasonDetailedResults.csv')
tourney_compact_all = pd.read_csv('data/MNCAATourneyCompactResults.csv')
tourney_detailed = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
teams = pd.read_csv('data/MTeams.csv')
seeds_all = pd.read_csv('data/MNCAATourneySeeds.csv')
slots_all = pd.read_csv('data/MNCAATourneySlots.csv')

In [3]:
# trim everything to start in 2003
min_year = 2003
regular_season_compact = regular_season_compact_all[regular_season_compact_all['Season'] >= min_year].reset_index(drop=True)
tourney_compact = tourney_compact_all[tourney_compact_all['Season'] >= min_year].reset_index(drop=True)
seeds = seeds_all[seeds_all['Season'] >= min_year].reset_index(drop=True)
slots = slots_all[slots_all['Season'] >= min_year].reset_index(drop=True)

## Look at data

In [4]:
regular_season_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2003,10,1104,68,1328,62,N,0
1,2003,10,1272,70,1393,63,N,0
2,2003,11,1266,73,1437,61,N,0
3,2003,11,1296,56,1457,50,N,0
4,2003,11,1400,77,1208,71,N,0


In [5]:
regular_season_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [6]:
tourney_compact.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2003,134,1421,92,1411,84,N,1
1,2003,136,1112,80,1436,51,N,0
2,2003,136,1113,84,1272,71,N,0
3,2003,136,1141,79,1166,73,N,0
4,2003,136,1143,76,1301,74,N,1


In [7]:
tourney_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [8]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2022
1,1102,Air Force,1985,2022
2,1103,Akron,1985,2022
3,1104,Alabama,1985,2022
4,1105,Alabama A&M,2000,2022


In [11]:
seeds.head()

,Season,Seed,TeamID
0,2003,W01,1328
1,2003,W02,1448
2,2003,W03,1393
3,2003,W04,1257
4,2003,W05,1280


In [12]:
slots.head()

,Season,Slot,StrongSeed,WeakSeed
0,2003,R1W1,W01,W16
1,2003,R1W2,W02,W15
2,2003,R1W3,W03,W14
3,2003,R1W4,W04,W13
4,2003,R1W5,W05,W12


## Feature engineering

Create: 
- games played
- regular season wins
- reg. season losses
- avg. game margin
- std. dev. game margin
- win %
- away/neutral wins
- home losses
- avg. points for
- avg. points against

In [17]:
regular_season_detailed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [180]:
# aggregate data for when team is the winning team
wins_q = '''
SELECT 
    Season, WTeamID AS TeamID, 
    COUNT() as num_wins,
    GROUP_CONCAT(WScore - LScore) as winning_margins,
    SUM(WScore) sum_score,
    SUM(LScore) sum_score_of_opponents,
    SUM(CASE WHEN WLoc == 'A' OR WLoc == 'N' THEN 1 ELSE 0 END) non_home_wins
FROM regular_season_detailed
GROUP BY 1, 2
'''
wins = sqldf(wins_q)
wins.head()

,Season,TeamID,num_wins,winning_margins,sum_score,sum_score_of_opponents,non_home_wins
0,2003,1102,12,"29,5,21,14,9,18,38,13,23,4,5,8",825,638,3
1,2003,1103,13,"4,7,33,5,1,18,1,16,11,2,1,15,8",1141,1019,4
2,2003,1104,17,"6,26,15,6,28,9,9,8,18,7,10,6,9,16,5,23,23",1270,1046,4
3,2003,1105,7,"19,7,4,1,18,34,8",556,465,2
4,2003,1106,13,"5,5,12,9,10,6,1,18,26,24,11,4,4",888,753,5


In [181]:
# aggregate data for when team is the losing team
losses_q = '''
SELECT 
    Season, LTeamID AS TeamID, 
    COUNT() as num_losses,
    GROUP_CONCAT(LScore - WScore) as losing_margins,
    SUM(LScore) sum_score,
    SUM(WScore) sum_score_of_opponents,
    SUM(CASE WHEN WLoc == 'A' THEN 1 ELSE 0 END) home_losses
FROM regular_season_detailed
GROUP BY 1, 2
'''
losses = sqldf(losses_q)
losses.head()

,Season,TeamID,num_losses,losing_margins,sum_score,sum_score_of_opponents,home_losses
0,2003,1102,16,"-18,-13,-17,-10,-15,-2,-2,-10,-32,-14,-7,-14,-...",778,958,4
1,2003,1103,14,"-3,-3,-2,-6,-8,-14,-15,-7,-16,-7,-16,-2,-4,-2",986,1091,5
2,2003,1104,11,"-2,-1,-9,-19,-17,-11,-19,-5,-4,-4,-13",670,774,2
3,2003,1105,19,"-28,-42,-13,-2,-22,-14,-10,-1,-1,-6,-5,-12,-31...",1310,1528,7
4,2003,1106,15,"-1,-12,-26,-16,-1,-9,-20,-6,-6,-1,-3,-19,-6,-1...",893,1032,4


In [192]:
# outer join wins and losses to get all teams W/L record each season
team_season_stats = wins.merge(losses, on=['Season', 'TeamID'], how='outer', suffixes=('_w', '_l'))
team_season_stats.fillna(0, inplace=True)

# typecast columns
team_season_stats[['num_wins','num_losses']] = team_season_stats[['num_wins','num_losses']].astype(int)
team_season_stats[['winning_margins','losing_margins']] = team_season_stats[['winning_margins','losing_margins']].astype(str)

# total games
team_season_stats['total_games'] = team_season_stats['num_wins'] + team_season_stats['num_losses']

# win pct.
team_season_stats['win_pct'] = team_season_stats['num_wins'] / team_season_stats['total_games']

# avg. points for
team_season_stats['avg_points_for'] = (team_season_stats['sum_score_w'] + team_season_stats['sum_score_l']) / team_season_stats['total_games']

# avg. points against
team_season_stats['avg_points_against'] = (team_season_stats['sum_score_of_opponents_w'] + team_season_stats['sum_score_of_opponents_l']) / team_season_stats['total_games']

# turn strings of winning and losing margins into one tuple
team_season_stats['differential_arr'] = [literal_eval(wm + ',' + lm) for wm, lm in zip(team_season_stats['winning_margins'], team_season_stats['losing_margins'])]

# avg. score differential
team_season_stats['avg_game_margin'] = [np.mean(arr) for arr in team_season_stats['differential_arr']]

# std. dev. of score differential
team_season_stats['std_game_margin'] = [np.std(arr) for arr in team_season_stats['differential_arr']]

In [194]:
team_season_stats_compact = team_season_stats[['Season', 'TeamID', 'total_games', 
                     'num_wins', 'num_losses', 'win_pct',
                     'avg_points_for', 'avg_points_against',
                     'avg_game_margin', 'std_game_margin',
                     'non_home_wins', 'home_losses']]
team_season_stats_compact.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses
0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,3.0,4.0
1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,4.0,5.0
2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0
3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,2.0,7.0
4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,5.0,4.0


Create:
- seed
- games vs. tourney teams
- wins vs. tourney teams (good wins)
- losses vs. tourney teams (good losses)
- away wins vs. tourney teams (tough wins)
- losses vs. non-tourney teams (bad losses)
- win % vs. tourney teams

In [222]:
# join game results with seeds
team_seeds_q = '''
SELECT 
    a.*, b.Seed WTeamSeed, c.Seed LTeamSeed
FROM regular_season_compact a
LEFT JOIN seeds b
    ON a.WTeamID = b.TeamID
    AND a.Season = b.Season
LEFT JOIN seeds c
    ON a.LTeamID = c.TeamID
    AND a.Season = c.Season
'''
team_seeds = sqldf(team_seeds_q)
# make sure both columns are strings
team_seeds[['WTeamSeed','LTeamSeed']] = team_seeds[['WTeamSeed','LTeamSeed']].astype(str)
team_seeds.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamSeed,LTeamSeed
0,2003,10,1104,68,1328,62,N,0,Y10,W01
1,2003,10,1272,70,1393,63,N,0,Z07,W03
2,2003,11,1266,73,1437,61,N,0,Y03,None
3,2003,11,1296,56,1457,50,N,0,None,None
4,2003,11,1400,77,1208,71,N,0,X01,None


In [214]:
wins_tourney_q = '''
SELECT
    Season, WTeamID AS TeamID, WTeamSeed AS Seed,
    SUM(CASE WHEN LTeamSeed != 'None' THEN 1 ELSE 0 END) wins_vs_tourney_teams,
    SUM(CASE WHEN LTeamSeed != 'None' AND WLoc != 'H' THEN 1 ELSE 0 END) away_wins_vs_tourney_teams
FROM team_seeds
GROUP BY 1, 2
'''
wins_tourney = sqldf(wins_tourney_q)
wins_tourney.head()

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams
0,2003,1102,None,1,0
1,2003,1103,None,1,1
2,2003,1104,Y10,5,1
3,2003,1105,None,1,0
4,2003,1106,None,1,0


In [221]:
losses_tourney_q = '''
SELECT
    Season, LTeamID AS TeamID, LTeamSeed AS Seed,
    SUM(CASE WHEN WTeamSeed != 'None' THEN 1 ELSE 0 END) losses_vs_tourney_teams,
    SUM(CASE WHEN WTeamSeed = 'None' THEN 1 ELSE 0 END) losses_vs_non_tourney_teams
FROM team_seeds
GROUP BY 1, 2
'''
losses_tourney = sqldf(losses_tourney_q)
losses_tourney.head()

,Season,TeamID,Seed,losses_vs_tourney_teams,losses_vs_non_tourney_teams
0,2003,1102,None,7,9
1,2003,1103,None,1,13
2,2003,1104,Y10,6,5
3,2003,1105,None,3,16
4,2003,1106,None,4,11


In [224]:
team_season_stats_tourney = wins_tourney.merge(losses_tourney, on=['Season', 'TeamID'], how='outer', suffixes=('_w', '_l'))
team_season_stats_tourney

,Season,TeamID,Seed_w,wins_vs_tourney_teams,away_wins_vs_tourney_teams,Seed_l,losses_vs_tourney_teams,losses_vs_non_tourney_teams
0,2003,1102,None,1.0,0.0,None,7.0,9.0
1,2003,1103,None,1.0,1.0,None,1.0,13.0
2,2003,1104,Y10,5.0,1.0,Y10,6.0,5.0
3,2003,1105,None,1.0,0.0,None,3.0,16.0
4,2003,1106,None,1.0,0.0,None,4.0,11.0
...,...,...,...,...,...,...,...,...
6887,2015,1363,NaN,NaN,NaN,None,5.0,23.0
6888,2021,1152,NaN,NaN,NaN,None,4.0,5.0
6889,2022,1175,NaN,NaN,NaN,None,0.0,16.0
6890,2022,1237,NaN,NaN,NaN,None,0.0,19.0


In [225]:
team_season_stats_vs_tourney_teams_q = '''
SELECT
    Season, TeamID,
    COALESCE(Seed_w, Seed_l) Seed,
    COALESCE(wins_vs_tourney_teams, 0) wins_vs_tourney_teams,
    COALESCE(away_wins_vs_tourney_teams, 0) away_wins_vs_tourney_teams,
    COALESCE(losses_vs_tourney_teams, 0) losses_vs_tourney_teams,
    COALESCE(losses_vs_non_tourney_teams, 0) losses_vs_non_tourney_teams
FROM team_season_stats_tourney
GROUP BY 1, 2
'''
team_season_stats_vs_tourney_teams = sqldf(team_season_stats_vs_tourney_teams_q)
team_season_stats_vs_tourney_teams.head()

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams
0,2003,1102,None,1.0,0.0,7.0,9.0
1,2003,1103,None,1.0,1.0,1.0,13.0
2,2003,1104,Y10,5.0,1.0,6.0,5.0
3,2003,1105,None,1.0,0.0,3.0,16.0
4,2003,1106,None,1.0,0.0,4.0,11.0


In [231]:
team_season_stats_vs_tourney_teams['games_vs_tourney_teams'] = \
    team_season_stats_vs_tourney_teams['wins_vs_tourney_teams'] + team_season_stats_vs_tourney_teams['losses_vs_tourney_teams']
team_season_stats_vs_tourney_teams['win_pct_vs_tourney_teams'] = \
    team_season_stats_vs_tourney_teams['wins_vs_tourney_teams'] / team_season_stats_vs_tourney_teams['games_vs_tourney_teams']

In [233]:
team_season_stats_vs_tourney_teams.head()

,Season,TeamID,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,win_pct_vs_tourney_teams
0,2003,1102,None,1.0,0.0,7.0,9.0,8.0,0.125000
1,2003,1103,None,1.0,1.0,1.0,13.0,2.0,0.500000
2,2003,1104,Y10,5.0,1.0,6.0,5.0,11.0,0.454545
3,2003,1105,None,1.0,0.0,3.0,16.0,4.0,0.250000
4,2003,1106,None,1.0,0.0,4.0,11.0,5.0,0.200000
